# Scroller

Scroller 有啥用？

让滚动动画更加顺滑。

Scroller 只追踪滚动的偏移量，不会自动地把这些位置信息应用到view上。

# Scroller的典型用法分析

```java
Scroller mScroller = new Scroller(mContext);

private void smoothScroll(int destX, int destY) {
        int scrollX = getScrollX();
        int deltaX = destX - scrollX;
        mScroller.startScroll(scrollX, 0, deltaX, 0, 500);
        invalidate();
    }

@Override
    public void computeScroll() {
        super.computeScroll();
        if (mScroller.computeScrollOffset()) {
            scrollTo(mScroller.getCurrX(), mScroller.getCurrY());
            invalidate();
        }
    }
```
上面是Scroller的最简单用法，可以看到Scroller其实只负责记录和计算滑动的xy坐标，并没有驱动View控件的滑动偏移，最终仍由继承的scrollTo/scrollBy方法实现，To和By后缀在Android中也很常见，命名统一规范对阅读源码很有帮助，其差异一眼便知。

View中的computeScroll方法是一个空实现，在源码中很多和draw相关的方法中被调用，Scroller的方法一般和invalidate等重绘方法出现，当View重绘时会调用computeScroll方法，只须在这里滑动到对应的位置即可。

Scroller 的 computeScrollOffset 负责判断滑动|Fling是否结束，可以看到下面的代码中Scroller处理了插值器，fling模式还处理了对应的滑动惯性，这里实质上是Scroller的核心作用：让滑动更加平滑。
```java
// Call this when you want to know the new location. If it returns true, the animation is not yet finished.

public boolean computeScrollOffset() {
        if (mFinished) {
            return false;
        }

        int timePassed = (int)(AnimationUtils.currentAnimationTimeMillis() - mStartTime);
    
        if (timePassed < mDuration) {
            switch (mMode) {
            case SCROLL_MODE:
                final float x = mInterpolator.getInterpolation(timePassed * mDurationReciprocal);
                mCurrX = mStartX + Math.round(x * mDeltaX);
                mCurrY = mStartY + Math.round(x * mDeltaY);
                break;
            case FLING_MODE:
                final float t = (float) timePassed / mDuration;
                final int index = (int) (NB_SAMPLES * t);
                float distanceCoef = 1.f;
                float velocityCoef = 0.f;
                if (index < NB_SAMPLES) {
                    final float t_inf = (float) index / NB_SAMPLES;
                    final float t_sup = (float) (index + 1) / NB_SAMPLES;
                    final float d_inf = SPLINE_POSITION[index];
                    final float d_sup = SPLINE_POSITION[index + 1];
                    velocityCoef = (d_sup - d_inf) / (t_sup - t_inf);
                    distanceCoef = d_inf + (t - t_inf) * velocityCoef;
                }

                mCurrVelocity = velocityCoef * mDistance / mDuration * 1000.0f;
                
                mCurrX = mStartX + Math.round(distanceCoef * (mFinalX - mStartX));
                // Pin to mMinX <= mCurrX <= mMaxX
                mCurrX = Math.min(mCurrX, mMaxX);
                mCurrX = Math.max(mCurrX, mMinX);
                
                mCurrY = mStartY + Math.round(distanceCoef * (mFinalY - mStartY));
                // Pin to mMinY <= mCurrY <= mMaxY
                mCurrY = Math.min(mCurrY, mMaxY);
                mCurrY = Math.max(mCurrY, mMinY);

                if (mCurrX == mFinalX && mCurrY == mFinalY) {
                    mFinished = true;
                }

                break;
            }
        }
        else {
            mCurrX = mFinalX;
            mCurrY = mFinalY;
            mFinished = true;
        }
        return true;
    }
```

当计算结果是true(滑动未结束)我们则需要通过View的scroll方法滑动到相应位置，同时再调用重绘方法以实现整个滑动循环。

# View系统配置参数

Android系统有很多有用的常量，比如判断双击的间隔时间，判断滑动的位移距离等等，使用这些常量可以让控件达到较好的交互效果。
```java
final ViewConfiguration vc = ViewConfiguration.get(context);
mTouchSlop = vc.getScaledTouchSlop();
mMinFlingVelocity = vc.getScaledMinimumFlingVelocity();
mMaxFlingVelocity = vc.getScaledMaximumFlingVelocity();
DisplayMetrics metric = context.getResources().getDisplayMetrics();
SCREEN_WIDTH = metric.widthPixels;
SCREEN_HEIGHT = metric.heightPixels;
```

# VelocityTracker 惯性滑动分析

VelocityTracker 也是一个追踪滑动速度的工具类，在ViewPager等滑动控件中很常见。

## 典型用法：获取抬起时的速度

```kotlin
 /**
* 速度追踪
*/
private var velocityTracker: VelocityTracker? = null
// 限制其最大和最小速度。因为速度过快和过慢，都会导致交互效果不佳
private val maxVelocity = ViewConfiguration.get(context).scaledMaximumFlingVelocity
private val minVelocity = ViewConfiguration.get(context).scaledMinimumFlingVelocity

     fun handleEvent(event: MotionEvent): Boolean {

            if (velocityTracker == null) {
                velocityTracker = VelocityTracker.obtain()
            }
            velocityTracker!!.addMovement(event)

            when (event.action) {
                MotionEvent.ACTION_DOWN -> {
                    // ...
                    return true
                }
                MotionEvent.ACTION_MOVE -> {
                    // ...
                    return true
                }
                MotionEvent.ACTION_UP -> {
                    // ...
                    // 计算手抬起时的速度
                    velocityTracker!!.computeCurrentVelocity(1000, maxVelocity.toFloat())
                    val velocityY = velocityTracker!!.yVelocity
                    if (abs(velocityY) > minVelocity) {
                        // 如果满足惯性滑动的最小速度
                        start(velocityY)
                    } else {
                        // ...
                    }
                    velocityTracker?.recycle()
                    velocityTracker = null
                    return true
                }
                else -> {
                }
            }
            return false
        }
```
上面的代码VelocityTracker延迟实例化，这是一个小优化方法，很多不一定用到的对象可以延迟生成。

addMovement用于追踪滑动数据，最终通过computeCurrentVelocity获得滑动速度，具体方法都是native实现，这里就不看了。

这里着重分析下VelocityTracker的复用机制。

# VelocityTracker的复用机制分析

```java
    private static final SynchronizedPool<VelocityTracker> sPool =
            new SynchronizedPool<VelocityTracker>(2);
    // ...
    static public VelocityTracker obtain() {
        VelocityTracker instance = sPool.acquire();
        return (instance != null) ? instance : new VelocityTracker(null);
    }
    // ...
    public void recycle() {
        if (mStrategy == null) {
            clear();
            sPool.release(this);
        }
    }
```
obtain和recycle方法要配对使用，第一次使用生成了一个实例，后面通过recycle回收到池中。

SynchronizedPool 是Android v4包下的池类，代码简单且优秀，很不错的学习参考。
```java
package androidx.core.util;


import androidx.annotation.NonNull;
import androidx.annotation.Nullable;

/**
 * Helper class for creating pools of objects. An example use looks like this:
 * <pre>
 * public class MyPooledClass {
 *
 *     private static final SynchronizedPool<MyPooledClass> sPool =
 *             new SynchronizedPool<MyPooledClass>(10);
 *
 *     public static MyPooledClass obtain() {
 *         MyPooledClass instance = sPool.acquire();
 *         return (instance != null) ? instance : new MyPooledClass();
 *     }
 *
 *     public void recycle() {
 *          // Clear state if needed.
 *          sPool.release(this);
 *     }
 *
 *     . . .
 * }
 * </pre>
 *
 */
public final class Pools {

    /**
     * Interface for managing a pool of objects.
     *
     * @param <T> The pooled type.
     */
    public interface Pool<T> {

        /**
         * @return An instance from the pool if such, null otherwise.
         */
        @Nullable
        T acquire();

        /**
         * Release an instance to the pool.
         *
         * @param instance The instance to release.
         * @return Whether the instance was put in the pool.
         *
         * @throws IllegalStateException If the instance is already in the pool.
         */
        boolean release(@NonNull T instance);
    }

    private Pools() {
        /* do nothing - hiding constructor */
    }

    /**
     * Simple (non-synchronized) pool of objects.
     *
     * @param <T> The pooled type.
     */
    public static class SimplePool<T> implements Pool<T> {
        private final Object[] mPool;

        private int mPoolSize;

        /**
         * Creates a new instance.
         *
         * @param maxPoolSize The max pool size.
         *
         * @throws IllegalArgumentException If the max pool size is less than zero.
         */
        public SimplePool(int maxPoolSize) {
            if (maxPoolSize <= 0) {
                throw new IllegalArgumentException("The max pool size must be > 0");
            }
            mPool = new Object[maxPoolSize];
        }

        @Override
        @SuppressWarnings("unchecked")
        public T acquire() {
            if (mPoolSize > 0) {
                final int lastPooledIndex = mPoolSize - 1;
                T instance = (T) mPool[lastPooledIndex];
                mPool[lastPooledIndex] = null;
                mPoolSize--;
                return instance;
            }
            return null;
        }

        @Override
        public boolean release(@NonNull T instance) {
            if (isInPool(instance)) {
                throw new IllegalStateException("Already in the pool!");
            }
            if (mPoolSize < mPool.length) {
                mPool[mPoolSize] = instance;
                mPoolSize++;
                return true;
            }
            return false;
        }

        private boolean isInPool(@NonNull T instance) {
            for (int i = 0; i < mPoolSize; i++) {
                if (mPool[i] == instance) {
                    return true;
                }
            }
            return false;
        }
    }

    /**
     * Synchronized) pool of objects.
     *
     * @param <T> The pooled type.
     */
    public static class SynchronizedPool<T> extends SimplePool<T> {
        private final Object mLock = new Object();

        /**
         * Creates a new instance.
         *
         * @param maxPoolSize The max pool size.
         *
         * @throws IllegalArgumentException If the max pool size is less than zero.
         */
        public SynchronizedPool(int maxPoolSize) {
            super(maxPoolSize);
        }

        @Override
        public T acquire() {
            synchronized (mLock) {
                return super.acquire();
            }
        }

        @Override
        public boolean release(@NonNull T element) {
            synchronized (mLock) {
                return super.release(element);
            }
        }
    }
}

```
定义Pool接口，SimplePool实现了简单的回收复用逻辑，注意是release的对象被回收到池中，SynchronizedPool在此基础上实现同步锁。嗯，这写法很Java。

# 滑动选择控件实现分析

之前写的滑动选择控件，实现思路是根据滑动的偏移量绘制item，绘制的item循环复用，item和实际选择项实现映射。

例如选择的item有100个，实际绘制的只有10个，我们只需根据滑动偏移量和item高度计算出映射的item显示即可。

通过Scroller和VelocityTracker实现了平滑的惯性滑动，通过简单的公式实现位置和缩放的计算实现缩放效果。

ScrollSelectView.kt

```kotlin
import android.annotation.SuppressLint
import android.content.Context
import android.graphics.Canvas
import android.graphics.Color
import android.graphics.Paint
import android.util.AttributeSet
import android.util.Log
import android.view.MotionEvent
import android.view.VelocityTracker
import android.view.View
import android.view.ViewConfiguration
import android.view.animation.DecelerateInterpolator
import android.widget.Scroller
import java.util.*
import kotlin.math.abs
import kotlin.math.min
import kotlin.math.pow


/**
 * by ganxiao
 */
class ScrollSelectView @JvmOverloads constructor(
    context: Context?,
    attrs: AttributeSet? = null,
    defStyleAttr: Int = 0
) : View(context, attrs, defStyleAttr) {
    /**
     * 默认宽度
     */
    private val defWidth = 80.dpToPx()

    /**
     * 默认高度
     */
    private val defHeight = 300.dpToPx()

    /**
     * 选择回调
     */
    var listener: OnValueChangeListener? = null

    /**
     * 布长
     */
    private var step = 1

    /**
     * 滑动的下标，真实值为该值对取值范围取余
     */
    private var index = 0

    /**
     * 选中行文字大小
     */
    private var textSize = 45.dpToPx()

    /**
     * 未选中行文字颜色
     */
    private val textNormalColor = Color.WHITE

    /**
     * 选中行文字颜色
     */
    private val textActiveColor = 0xffFFB27D.toInt()

    private val mPaint = Paint(Paint.ANTI_ALIAS_FLAG)

    /**
     * 最小值
     */
    private var minValue = 0

    /**
     * 最大值
     */
    private var maxValue = 99

    /**
     * 滑动时手指的Y轴移动距离
     */
    private var moveY = 0f

    /**
     * 每一行文字内容占据的高度
     */
    private var lineHeight = 0f

    /**
     * 高亮的数字上还要绘制的个数
     */
    private val topN = 800

    /**
     * 高亮的数字上显示的个数
     */
    private val topNShow = 2

    /**
     * 缩放系数
     */
    private val zoomFactor = 2f

    /**
     * 放置选中 value 的数组
     */
    private val showValueList: MutableList<Int> = ArrayList()

    /**
     * 选择部份是否包含最大值
     */
    private var includeMax = false

    /**
     * 滑动帮助类
     */
    private val scrollHelper = ScrollHelper()

    init {
        refreshShowValueList()
    }

    fun debug(msg: String) {
        Log.d("ScrollSelectView", msg)
    }

    override fun onMeasure(widthMeasureSpec: Int, heightMeasureSpec: Int) {
        setMeasuredDimension(
            resolveSize(defWidth, widthMeasureSpec),
            resolveSize(defHeight, heightMeasureSpec)
        )
    }

    @SuppressLint("ClickableViewAccessibility")
    override fun onTouchEvent(event: MotionEvent): Boolean {
        val result = scrollHelper.handleEvent(event)
        return if (result)
            result
        else
            super.onTouchEvent(event)
    }

    override fun onDraw(canvas: Canvas) {
        lineHeight = height / (1f + 2f * topNShow)
        mPaint.textSize = textSize.toFloat()
        var startY = moveY + (0 - lineHeight * (topN - topNShow))
        debug("moveY: " + moveY)
        for (i in 0 until 1 + topN * 2) {
            val centerY = startY + lineHeight / 2f
            val drawIndex = index + i - topN
            val drawValue = getValueFromIndex(drawIndex)
            if (centerY > lineHeight * topNShow && centerY < lineHeight * (topNShow + 1)) {
                mPaint.color = textActiveColor
            } else {
                mPaint.color = textNormalColor
            }
            canvas.save()
            // val scale = calScale(centerY)
            val scale = 1f
            canvas.scale(scale, scale, 0f, centerY)
            val text = drawValue.format02d()
            mPaint.textAlign = Paint.Align.CENTER
            val fontMetrics = mPaint.fontMetrics
            val tmp = abs(fontMetrics.top) + abs(fontMetrics.bottom) / 2f - abs(fontMetrics.bottom)
            canvas.drawText(text, width * 0.5f / scale, centerY + tmp, mPaint)
            canvas.restore()
            startY += lineHeight
        }
    }

    /**
     * 缩放公式
     */
    private fun calScale(y: Float): Float {
        var result = 0.2f
        if (y <= 0 || y > height) {
            return result
        }
        val t = y / height
        result = (1 - (zoomFactor * (t - 0.5f).toDouble()).pow(2.0)).toFloat()
        val minScale = 0.7f
        return min(minScale, result)
    }

    var value: Int
        get() = getValueFromIndex(index)
        set(value) {
            for (i in showValueList.indices) {
                if (value == showValueList[i]) {
                    index = i
                    refreshShowValueList()
                    invalidate()
                    return
                }
            }
        }

    private fun refreshShowValueList() {
        showValueList.clear()
        if (includeMax) {
            var value = minValue
            while (value <= maxValue) {
                showValueList.add(value)
                value += step
            }
        } else {
            var value = minValue
            while (value < maxValue) {
                showValueList.add(value)
                value += step
            }
        }
    }

    /**
     * 根据当前的偏移量确定用户选择的 value
     */
    private fun handleValueSelect() {
        var absY = abs(moveY)
        val direction = if (moveY > 0) -1 else 1
        if (absY > lineHeight / 2f) {
            var moveIndex = 1
            absY -= lineHeight / 2f
            moveIndex += (absY / lineHeight).toInt()
            index += moveIndex * direction
            listener?.onValueChange(value)
        }
        invalidate()
        moveY = 0f
    }

    private fun getValueFromIndex(i: Int): Int {
        var i = i
        if (maxValue <= minValue) {
            return minValue
        }
        if (showValueList.size == 0) {
            return minValue
        }
        if (i < 0) {
            i = showValueList.size + i % showValueList.size
        }
        val value = i % showValueList.size
        return showValueList[value]
    }

    inner class ScrollHelper : Runnable {

        /**
         * 速度追踪
         */
        private var velocityTracker: VelocityTracker? = null
        private val maxVelocity = ViewConfiguration.get(context).scaledMaximumFlingVelocity
        private val minVelocity = ViewConfiguration.get(context).scaledMinimumFlingVelocity
        private val scroller = Scroller(context, DecelerateInterpolator())
        private var startY = 0f
        private val maxScrollY = 405.dpToPx()
        private var preMoveY = 0f

        fun handleEvent(event: MotionEvent): Boolean {

            if (velocityTracker == null) {
                velocityTracker = VelocityTracker.obtain()
            }
            velocityTracker!!.addMovement(event)

            when (event.action) {
                MotionEvent.ACTION_DOWN -> {
                    scrollHelper.stop()
                    startY = event.y
                    return true
                }
                MotionEvent.ACTION_MOVE -> {
                    moveY = event.y - startY
                    invalidate()
                    return true
                }
                MotionEvent.ACTION_UP -> {
                    moveY = event.y - startY
                    // 计算手抬起时的速度
                    velocityTracker!!.computeCurrentVelocity(1000, maxVelocity.toFloat())
                    val velocityY = velocityTracker!!.yVelocity
                    debug("velocityY: $velocityY")
                    if (abs(velocityY) > minVelocity) {
                        start(velocityY)
                    } else {
                        handleValueSelect()
                    }
                    velocityTracker?.recycle()
                    velocityTracker = null
                    return true
                }
                else -> {
                }
            }
            return false
        }

        /**
         * 开始滚动
         */
        private fun start(velocityY: Float) {
            preMoveY = moveY
            // 先停止上一次的滚动
            if (!scroller.isFinished) {
                scroller.abortAnimation()
            }

            // 触发 fling
            scroller.fling(
                0, 0,
                0, velocityY.toInt(),
                0, 0,
                -maxScrollY, maxScrollY
            )

            // 开启滚动循环, 根据scroller的偏移量刷新界面
            post(this)
        }

        override fun run() {
            moveY = preMoveY + scroller.currY.toFloat()
            debug("run moveY: $moveY")
            invalidate()

            if (scroller.computeScrollOffset()) {
                post(this)
                return
            } else {
                // 滚动已经结束
                handleValueSelect()
            }
        }

        private fun stop() {
            if (!scroller.isFinished)
                scroller.abortAnimation()
        }

    }

    interface OnValueChangeListener {
        fun onValueChange(value: Int)
    }

}
```